In [1]:
import ruptures as rpt
import gzip
import numpy as np
import time
import pandas as pd

In [2]:
signal_df = pd.read_csv('https://github.com/tdhock/LOPART-paper/raw/master/data-for-LOPART-signals.csv.gz', compression='gzip')
seqs = tuple(signal_df.groupby('sequenceID'))

label_df = pd.read_csv('https://github.com/tdhock/LOPART-paper/raw/master/data-for-LOPART-labels.csv.gz', compression='gzip')
labels = tuple(label_df.groupby('sequenceID'))

In [3]:
def get_data(i, seqs, labels):
    # sequence
    sequence = seqs[i][1]['logratio'].to_numpy()
    sequence = np.append([0], sequence)

    # labels
    lab_df = labels[i][1]

    # get label sets
    lab_df_1, lab_df_2 = lab_df[lab_df['fold'] == 1], lab_df[lab_df['fold'] == 2]

    pos_lab_df_1, pos_lab_df_2 = lab_df_1[lab_df_1['changes'] == 1], lab_df_2[lab_df_2['changes'] == 1]
    neg_lab_df_1, neg_lab_df_2 = lab_df_1[lab_df_1['changes'] == 0], lab_df_2[lab_df_2['changes'] == 0]

    neg_start_1, neg_end_1 = neg_lab_df_1['start'].to_numpy(), neg_lab_df_1['end'].to_numpy()
    pos_start_1, pos_end_1 = pos_lab_df_1['start'].to_numpy(), pos_lab_df_1['end'].to_numpy()
    neg_start_2, neg_end_2 = neg_lab_df_2['start'].to_numpy(), neg_lab_df_2['end'].to_numpy()
    pos_start_2, pos_end_2 = pos_lab_df_2['start'].to_numpy(), pos_lab_df_2['end'].to_numpy()

    return sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2

In [4]:
def get_cumsum(sequence):
    y = np.cumsum(sequence)
    z = np.cumsum(np.square(sequence))

    y = np.append([0], y)
    z = np.append([0], z)

    return y, z

In [5]:
def L(start, end, y, z):
    _y = y[end+1] - y[start]
    _z = z[end+1] - z[start]
    return _z - np.square(_y)/(end-start+1)

In [6]:
def trace_back(tau_star):
    tau = tau_star[-1]
    chpnt = np.array([len(tau_star)], dtype=int)
    while tau > 0:
        chpnt = np.append(tau, chpnt)
        tau = tau_star[tau-1]
    return np.append(0, chpnt)

In [7]:
def opart(lda, sequence):

    # cumsum vector:
    y, z = get_cumsum(sequence)

    # length of sequence
    sequence_length = len(sequence)-1

    # Set up
    C = np.zeros(sequence_length + 1)
    C[0] = -lda

    # Get tau_star
    tau_star = np.zeros(sequence_length+1, dtype=int)
    for t in range(1, sequence_length+1):

        # get set of possible value
        V = C[:t] + lda + L(1 + np.arange(t), t, y, z)

        # get optimal tau from set V
        last_chpnt = np.argmin(V)

        # update C_i
        C[t] = V[last_chpnt]

        # update tau_star
        tau_star[t] = last_chpnt

    # get set of changepoints
    set_of_chpnt = trace_back(tau_star[1:])

    return set_of_chpnt

In [8]:
start_time = time.time()

for i in range(20):
    sequence, _, _, _, _, _, _, _, _ = get_data(i, seqs=seqs, labels=labels)
    pelt_result = rpt.Pelt(model="l2", min_size=1, jump=1).fit(sequence[1:]).predict(pen=1)
    print("Sequence:%2s \t Pelt: %20s" % (i+1, pelt_result))

end_time = time.time()
print("\nTotal time (seconds):", end_time - start_time)

Sequence: 1 	 Pelt: [187, 437, 460, 474]
Sequence: 2 	 Pelt:            [86, 155]
Sequence: 3 	 Pelt:             [33, 79]
Sequence: 4 	 Pelt:            [71, 163]
Sequence: 5 	 Pelt:            [28, 118]
Sequence: 6 	 Pelt:           [105, 199]
Sequence: 7 	 Pelt:           [127, 188]
Sequence: 8 	 Pelt:                [256]
Sequence: 9 	 Pelt:           [139, 223]
Sequence:10 	 Pelt:            [14, 170]
Sequence:11 	 Pelt:           [243, 414]
Sequence:12 	 Pelt:           [135, 303]
Sequence:13 	 Pelt:       [24, 175, 495]
Sequence:14 	 Pelt:            [19, 178]
Sequence:15 	 Pelt:      [487, 513, 535]
Sequence:16 	 Pelt:           [106, 199]
Sequence:17 	 Pelt:            [81, 188]
Sequence:18 	 Pelt:           [123, 333]
Sequence:19 	 Pelt:            [68, 223]
Sequence:20 	 Pelt:       [64, 122, 187]

Total time (seconds): 28.553200006484985


In [9]:
start_time = time.time()

for i in range(20):
    sequence, _, _, _, _, _, _, _, _ = get_data(i, seqs=seqs, labels=labels)
    my_result = list(opart(1, sequence)[1:])
    print("Sequence:%2s \t Mine: %20s" % (i+1, my_result))

end_time = time.time()
print("\nTotal time (seconds):", end_time - start_time)

Sequence: 1 	 Mine: [187, 437, 460, 474]
Sequence: 2 	 Mine:            [86, 155]
Sequence: 3 	 Mine:             [33, 79]
Sequence: 4 	 Mine:            [71, 163]
Sequence: 5 	 Mine:            [28, 118]
Sequence: 6 	 Mine:           [105, 199]
Sequence: 7 	 Mine:           [127, 188]
Sequence: 8 	 Mine:                [256]
Sequence: 9 	 Mine:           [139, 223]
Sequence:10 	 Mine:            [14, 170]
Sequence:11 	 Mine:           [243, 414]
Sequence:12 	 Mine:           [135, 303]
Sequence:13 	 Mine:       [24, 175, 495]
Sequence:14 	 Mine:            [19, 178]
Sequence:15 	 Mine:      [487, 513, 535]
Sequence:16 	 Mine:           [106, 199]
Sequence:17 	 Mine:            [81, 188]
Sequence:18 	 Mine:           [123, 333]
Sequence:19 	 Mine:            [68, 223]
Sequence:20 	 Mine:       [64, 122, 187]

Total time (seconds): 0.2806129455566406
